# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [1]:
df_aleatorio = pd.read_csv("instancias_aleatorias/complejidad/indice.csv")

In [6]:
df_aleatorio_agm_ins = pd.read_csv("instancias_aleatorias/agm-ins/indice.csv")

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

- **VMC** : vecino mas cercano
- **AGM** : arbol generador minimo
- **I** : incersion
- **tabu falta** : tabu

In [3]:
def correr_experimento(metodo, archivo_instancia,parametros_tabu):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    cmds = ["../tp2", metodo] + parametros_tabu
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(cmds , stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(f"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo = process.stdout.read() 
    #print(process.stdout.read()) # resultado del metodo
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    costo = costo.split(" ")[1]
    costo = costo.split("\n")[0]
    res = (tiempo_de_ejecucion,float(costo))
    
    return res;




## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [4]:
experimentos = [];
experimentos

[]

## Experimento aleatorio


In [13]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM", fila["archivo"],[str(0),str(0),str(0),str(0)]]);
    experimentos.append([fila["dataset"], fila["n"],"VMC", fila["archivo"],[str(0),str(0),str(0),str(0)]]);
    experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"],[str(0),str(0),str(0),str(0)]]);    

## Experimento AGM vs INS Aleatorio 

In [9]:
for i in range(0, df_aleatorio_agm_ins.shape[0]):
    fila = df_aleatorio_agm_ins.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"VMC", fila["archivo"],[str(0),str(0),str(0),str(0)]]);
    experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"],[str(0),str(0),str(0),str(0)]]);
    experimentos.append([fila["dataset"], fila["n"],"AGM", fila["archivo"],[str(0),str(0),str(0),str(0)]]);
    

## AGM INS VMC INSTANCIAS EUCLIDIANAS

In [5]:
data = "berlin52"
n = 52
archivo = "instancias_salida/burma14.txt"
experimentos.append([data, n ,"AGM", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"VMC", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"I", archivo,[str(0),str(0),str(0),str(0)]]);


data = "rd100"
n = 100
archivo = "instancias_salida/rd100.txt"
experimentos.append([data, n ,"AGM", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"VMC", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"I", archivo,[str(0),str(0),str(0),str(0)]]);


data = "ch150"
n = 150
archivo = "instancias_salida/ch150.txt"
experimentos.append([data, n ,"AGM", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"VMC", archivo,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"I", archivo,[str(0),str(0),str(0),str(0)]]);

## CORRER EXPERIMENTOS

In [6]:
columnas = ["dataset", "n", "metodo", "tiempo","costo","largolista","max_iter","rang_iter","percentage"];


filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    costos = []
    resultados = []
    for i in range(0, T):
        exp = correr_experimento(experimento[2], experimento[3],experimento[4])
        t = exp[0]
        c = exp[1]
        costos.append(c)
        tiempos.append(t);
    costo = np.median(costos)
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo,costo,
                  experimento[4][0],experimento[4][1],experimento[4][2],experimento[4][3]]);
print('fin :)')

'Experimento: 9/9'

fin :)


In [7]:
df_resultados = pd.DataFrame(filas, columns=columnas);

# df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [9]:
df_resultados

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,berlin52,52,AGM,0.027351,4003.0,0,0,0,0
1,berlin52,52,VMC,0.015940,4048.0,0,0,0,0
2,berlin52,52,I,0.947909,5774.0,0,0,0,0
3,rd100,100,AGM,0.546651,10467.0,0,0,0,0
4,rd100,100,VMC,0.395078,9938.0,0,0,0,0
5,rd100,100,I,346.227000,14193.0,0,0,0,0
6,ch150,150,AGM,1.057410,9315.0,0,0,0,0
7,ch150,150,VMC,1.025020,8191.0,0,0,0,0
8,ch150,150,I,1325.550000,13855.0,0,0,0,0


In [10]:
df_viejo = pd.read_csv("resultados/resultado.csv")

In [11]:
df_viejo

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,dataset_aleatorias_agm_ins,50,AGM,0.152634,4178.0,0,0,0,0
1,dataset_aleatorias_agm_ins,50,VMC,0.106869,1722.0,0,0,0,0
2,dataset_aleatorias_agm_ins,50,I,50.802100,3351.0,0,0,0,0
3,dataset_aleatorias_agm_ins,100,AGM,0.486455,7509.0,0,0,0,0
4,dataset_aleatorias_agm_ins,100,VMC,0.365159,2503.0,0,0,0,0
5,dataset_aleatorias_agm_ins,100,I,355.472000,6619.0,0,0,0,0
6,dataset_aleatorias_agm_ins,150,AGM,1.468770,10093.0,0,0,0,0
7,dataset_aleatorias_agm_ins,150,VMC,1.014140,2804.0,0,0,0,0
8,dataset_aleatorias_agm_ins,150,I,1328.920000,5901.0,0,0,0,0


In [12]:
df_nuevo = df_viejo.append(df_resultados, ignore_index = True)
df_nuevo

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,dataset_aleatorias_agm_ins,50,AGM,0.152634,4178.0,0,0,0,0
1,dataset_aleatorias_agm_ins,50,VMC,0.106869,1722.0,0,0,0,0
2,dataset_aleatorias_agm_ins,50,I,50.802100,3351.0,0,0,0,0
3,dataset_aleatorias_agm_ins,100,AGM,0.486455,7509.0,0,0,0,0
4,dataset_aleatorias_agm_ins,100,VMC,0.365159,2503.0,0,0,0,0
5,dataset_aleatorias_agm_ins,100,I,355.472000,6619.0,0,0,0,0
6,dataset_aleatorias_agm_ins,150,AGM,1.468770,10093.0,0,0,0,0
7,dataset_aleatorias_agm_ins,150,VMC,1.014140,2804.0,0,0,0,0
8,dataset_aleatorias_agm_ins,150,I,1328.920000,5901.0,0,0,0,0
9,berlin52,52,AGM,0.027351,4003.0,0,0,0,0


In [13]:
df_nuevo.to_csv("resultados/resultado.csv", index=False, header=True);